<a href="https://colab.research.google.com/github/TAbdullah-T/T5-SAD/blob/main/Deep%20Learning/Project/5_Mini_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading the Dataset

In [1]:
!git clone https://github.com/TariqAlhathloul/GTS_CNN.git
!pip install keras-tuner --upgrade
!pip install tensorflow

Cloning into 'GTS_CNN'...
remote: Enumerating objects: 51955, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 51955 (delta 8), reused 19 (delta 4), pack-reused 51930 (from 1)
Receiving objects: 100% (51955/51955), 305.76 MiB | 31.99 MiB/s, done.
Resolving deltas: 100% (8/8), done.
Updating files: 100% (51891/51891), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import os
from PIL import Image
#import tensorflow and keras
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
import kerastuner as kt

from sklearn.model_selection import train_test_split

<ipython-input-2-35c3774afa8b>:13: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [3]:
train_csv_path = 'GTS_CNN/DataSet/Train.csv'
test_csv_path  = 'GTS_CNN/DataSet/Test.csv'

base_path = 'GTS_CNN/DataSet/'

img_width, img_height = 120, 120

def load_and_resize_image(filepath):
    full_path = os.path.join(base_path, filepath)
    image = Image.open(full_path).convert('RGB')
    image = image.resize((img_width, img_height))
    return np.array(image)

# Load CSV files
train_df = pd.read_csv(train_csv_path)
test_df = pd.read_csv(test_csv_path)

# Load training data
X_train = []
y_train = []
for _, row in train_df.iterrows():
    image_path = row['Path']
    class_id = row['ClassId']
    X_train.append(load_and_resize_image(image_path))
    y_train.append(class_id)

# Convert to numpy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)

# Load test data
X_test = []
y_test = []
for _, row in test_df.iterrows():
    image_path = row['Path']
    class_id = row['ClassId']
    X_test.append(load_and_resize_image(image_path))
    y_test.append(class_id)
# Convert to numpy array
X_test = np.array(X_test)
y_test = np.array(y_test)


In [4]:
print(f'X_train shape: {X_train.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'X_test shape: {y_test.shape}')

X_train shape: (39209, 120, 120, 3)
y_train shape: (39209,)
X_test shape: (12630, 120, 120, 3)
X_test shape: (12630,)


## Data Agumentation

In [5]:
datagen = ImageDataGenerator(
    rescale = 1/255,
    rotation_range = 15,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    horizontal_flip = True,
    zoom_range = 0.2,
)
datagen.fit(X_train)

# Model Building

In [9]:
model = Sequential([
    #Block 1 : 32 units
    Conv2D(16,(3,3), activation='relu',padding='same', input_shape=(120,120,3)),
    BatchNormalization(),
    Dropout(0.5),
    Conv2D(16,(3,3), activation='relu', padding='same'),
    BatchNormalization(),
    Dropout(0.5),
    MaxPooling2D(2,2),

    #Block 2 : 64 units
    Conv2D(32,(3,3), activation='relu', padding='same'),
    BatchNormalization(),
    Dropout(0.5),
    Conv2D(32,(3,3), activation='relu', padding='same'),
    BatchNormalization(),
    Dropout(0.5),
    MaxPooling2D(2,2),


    # #Block 3 : 64 units
    Conv2D(64,(3,3), activation='relu', padding='same'),
    BatchNormalization(),
    Dropout(0.5),

    # Classifiction head
    Flatten(),
    # Dense(32, activation='relu'),
    # BatchNormalization(),
    Dense(43, activation='softmax')
])

model.compile(optimizer= 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

# Data Spliting and Training

In [7]:
X_train_, X_val, y_train_, y_val = train_test_split(X_train,y_train, test_size=0.2, random_state=0,shuffle=True)
X_train_.shape, X_val.shape

((31367, 120, 120, 3), (7842, 120, 120, 3))

In [10]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [11]:
early_stoping = EarlyStopping(monitor = 'val_accuracy', patience = 5 )

history = model.fit(datagen.flow(X_train_,y_train_,batch_size=64),
                    batch_size= 100,
                    epochs = 45,
                    validation_data = (X_val/255,y_val),
                    # callbacks=[early_stoping],
                    verbose=1)

Epoch 1/100
491/491 ━━━━━━━━━━━━━━━━━━━━ 119s 218ms/step - accuracy: 0.1306 - loss: 21.8180 - val_accuracy: 0.1651 - val_loss: 7.4457
Epoch 2/100
491/491 ━━━━━━━━━━━━━━━━━━━━ 99s 198ms/step - accuracy: 0.4325 - loss: 1.8998 - val_accuracy: 0.2855 - val_loss: 5.2236
Epoch 3/100
491/491 ━━━━━━━━━━━━━━━━━━━━ 99s 199ms/step - accuracy: 0.5163 - loss: 1.5638 - val_accuracy: 0.2999 - val_loss: 6.6016
Epoch 4/100
491/491 ━━━━━━━━━━━━━━━━━━━━ 99s 199ms/step - accuracy: 0.5852 - loss: 1.3161 - val_accuracy: 0.4138 - val_loss: 4.2525
Epoch 5/100
491/491 ━━━━━━━━━━━━━━━━━━━━ 98s 198ms/step - accuracy: 0.6510 - loss: 1.0885 - val_accuracy: 0.4200 - val_loss: 4.4345
Epoch 6/100
491/491 ━━━━━━━━━━━━━━━━━━━━ 99s 198ms/step - accuracy: 0.7126 - loss: 0.8896 - val_accuracy: 0.5597 - val_loss: 2.6102
Epoch 7/100
491/491 ━━━━━━━━━━━━━━━━━━━━ 98s 197ms/step - accuracy: 0.7546 - loss: 0.7761 - val_accuracy: 0.4543 - val_loss: 4.0296
Epoch 8/100
491/491 ━━━━━━━━━━━━━━━━━━━━ 98s 197ms/step - accuracy: 0.7824

KeyboardInterrupt: 

In [12]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label="train_loss")
plt.plot(history.history['val_loss'], label='val_loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(history.history['accuracy'], label='training_accuracy')
plt.plot(history.history['val_accuracy'], label='validation accuracy')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Accuacry')
plt.show()

NameError: name 'history' is not defined

# Model evaluation

In [16]:
model.evaluate(X_test/255, y_test)

395/395 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8305 - loss: 0.8005


[0.803300142288208, 0.8319873213768005]

# Hyper-Paramters Tunning  

In [ ]:
!pip install keras-tuner --upgrade

In [ ]:

def model_builder(hp):
    # Define specific filter values
    filter_values = hp.Choice([8, 16, 32, 64])

    activation_choice = hp.Choice('activation_function', ['relu', 'leaky_relu', 'tanh'])

    model = Sequential([
        Conv2D(filter_values, kernel_size=(3, 3), activation=activation_choice, input_shape=(120, 120, 3)),
        MaxPooling2D(2, 2),
        Conv2D(filter_values, kernel_size=(3, 3), activation=activation_choice),
        MaxPooling2D(2, 2),
        Conv2D(filter_values, kernel_size=(3, 3), activation=activation_choice),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(units=43, activation='softmax'),
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

def build_datagen(hp):
    return ImageDataGenerator(
        rotation_range=hp.Int('rotation_range', min_value=0, max_value=90, step=10),
        width_shift_range=hp.Float('width_shift_range', min_value=0.0, max_value=0.3, step=0.05),
        height_shift_range=hp.Float('height_shift_range', min_value=0.0, max_value=0.3, step=0.05),
        zoom_range=hp.Float('zoom_range', min_value=0.0, max_value=0.3, step=0.05),
        horizontal_flip=hp.Boolean('horizontal_flip'),
        rescale=1/255
    )

def train_test_model(hp):
    datagen = build_datagen(hp)
    datagen.fit(X_train)  # Make sure X_train is defined
    model = model_builder(hp)
    model.fit(
        datagen.flow(X_train, y_train, batch_size=64),
        validation_data=datagen.flow(X_val, y_val, batch_size=64),  # Ensure X_val and y_val are defined
        epochs=10, verbose=0
    )
    return model

tuner = kt.RandomSearch(
    model_builder,
    objective='val_accuracy',
    max_trials=3,
    executions_per_trial=3,
    directory='auto_tune_aug'
    # project_name='augmentation_tuning'
)

tuner.search()